In [2]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import numpy as np
import re
import pickle
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [25]:
#create dictionary for results, which will be used for further calculations
r_keys = ['L', 'W', 'L\xa0(1 OT)', 'L\xa0(2 OT)', 'W\xa0(2 OT)','W\xa0(1 OT)', 'W\xa0(3 OT)', 'L\xa0(3 OT)', 'W\xa0(4 OT)', 'L\xa0(4 OT)']
r_wins = [0,1,0,0,1,1,1,0,1,0] #1 for win
r_OT = [0,0,1,2,2,1,3,3,4,4] #overtimes played
r_MP = [40,40,45,50,50,45,55,55,60,60] #minutes played

results_dict = dict(zip(r_keys,zip(r_wins,r_OT,r_MP)))

# Data Cleaning and Feature Engineering

## Data Cleaning

In [ ]:
team_lookup = pickle.load(open('teams_lookup.pickle', "rb" ))

In [3]:
#column names for dataframes
combo_col_w_blank = ['url','Team','Date','Court','Opponent','Result','Tm','Opp','FG','FGA','FG%','3P','3PA','3P%','FT','FTA',
           'FT%','ORB','TRB','AST','STL','BLK','TOV','PF','blank','FG_O','FGA_O','FG%_O','3P_O','3PA_O','3P%_O','FT_O',
           'FTA_O','FT%_O','ORB_O','TRB_O','AST_O','STL_O','BLK_O','TOV_O','PF_O','url2','Team2','Date2','Court2','Opponent2','Result2','Tm2','Opp2','ORtg','DRtg','Pace','FTr','3PAr','TS%','TRB%',
           'AST%','STL%','BLK%','blank1','OeFG%','OTOV%','ORB%','OFT/FGA','blank2','DeFG%','DTOV%','DRB%','DFT/FGA']
num_cols = ['Tm','Opp','FG','FGA','FG%','3P','3PA','3P%','FT','FTA',
           'FT%','ORB','TRB','AST','STL','BLK','TOV','PF','FG_O','FGA_O','FG%_O','3P_O','3PA_O','3P%_O','FT_O',
           'FTA_O','FT%_O','ORB_O','TRB_O','AST_O','STL_O','BLK_O','TOV_O','PF_O','ORtg','DRtg','Pace','FTr','3PAr','TS%','TRB%',
           'AST%','STL%','BLK%','OeFG%','OTOV%','ORB%','OFT/FGA','DeFG%','DTOV%','DRB%','DFT/FGA']

In [18]:
def clean_gamelog(combo_log, col_names_wb = combo_col_w_blank,
                  col_names = num_cols):
    '''
    Takes in gamelog and cleans the data
    input: gamelog as dataframe and stat_type. stat_type can be basic or advanced
    output: dataframe with clean gamelog
    '''
    games = pd.DataFrame.from_dict(combo_log, orient = "index")
    games.columns = col_names_wb
    games['Date'] = pd.to_datetime(games['Date'])
    games = games.drop(columns=['blank','url2','Team2','Date2','Court2','Opponent2','Result2','Tm2','Opp2','blank1','blank2'])
    for i in col_names:
        games[i] = pd.to_numeric(games[i])
    return games

In [23]:
#adding columns to help with further data manipulation
def helper_columns(combo_log,team_lookup):
    '''
    this helps to clean data. update this here
    games = gamelog information by team ______
    columns = names of columns in the dataframe
    team_lookup = to get the right name
    
    output: dataframe cleaned!
    with columns we want
    
    '''
    hot_court = pd.get_dummies(combo_log.Court)
    combo_log['Home'],combo_log['Away'] = hot_court[""],hot_court["@"]
    combo_log['Wins'] = [results_dict[x][0] for x in combo_log.Result]
    combo_log['OT'] = [results_dict[x][1] for x in combo_log.Result]
    combo_log['MP']= [results_dict[x][2] for x in combo_log.Result]
    return combo_log

In [8]:
def D1_only(combo_log, team_lookup):
    '''
    Removes teams who are not in D1 
    
    '''
    #create lookup for D1 feature gen
    #D1 = indicate if opponent is in Division 1 or not (from the data collection, only D1 teams included for gamelog but they sometimes play non-D1 schools)

    team_df = pd.DataFrame.from_dict(team_lookup, orient = 'index')
    D1 = dict(zip(team_df.loc[:,0],np.repeat(1,len(team_df.loc[:,0]))))

    #removing all non-D1 games
    combo_log['D1'] = [D1.get(x) or 0 for x in combo_log['Opponent']]
    combo_log = combo_log[combo_log.D1 == 1]
    
    return combo_log

## Additional columns

In [9]:
#calculate various spreads

def calc_spreads(df):
    df['P_S'] = df.eval('Tm - Opp')
    df['FG_S'] = df.eval('FG - FG_O')
    df['FGA_S'] = df.eval('FGA - FGA_O')
    df['FG%_S'] = df['FG%'] - df['FG%_O']
    df['3P_S'] = df['3P'] - df['3P_O']
    df['3PA_S'] = df['3PA'] - df['3PA_O']
    df['3P%_S'] = df['3P%'] - df['3P%_O']
    df['FT_S'] = df.eval('FT - FT_O')
    df['FTA_S'] = df.eval('FTA - FTA_O')
    df['FT%_S'] = df['FT%'] - df['FT%_O']
    df['TRB_S'] = df.eval('TRB- TRB_O')
    df['AST_S'] = df.eval('AST - AST_O')
    df['STL_S'] = df.eval('STL - STL_O')
    df['BLK_S'] = df.eval('BLK - BLK_O')
    df['TOV_S'] = df.eval('TOV - TOV_O')
    df['PF_S'] = df.eval('PF - PF_O')
    df['FTr_S'] = df.eval('FTr - FT_O/FGA_O') 
    df['3PAr_S'] = df['3PA']/df['FG'] - df['3PA_O']/df['FG_O']
    df['TS%_S'] = df['TS%'] - df.eval('Opp/(2*(FGA_O+0.44*FTA_O))')
    df['TRB%_S'] = 2*df['TRB%'] - 1
    df['AST%_S'] = df.eval('(AST - AST_O)/Pace')
    df['STL%_S'] = df.eval('(STL - STL_O)/Pace')
    df['BLK%_S'] = df.eval('(BLK - BLK_O)/Pace')
    return df

## Preparing clean data file

In [10]:
def agg_df(games):
    '''
    This will compute the running stats for teams, with season-to-date info prior to each game
    '''
    df = pd.DataFrame(games['url'])
    df['Team'] = [team_lookup[x][0] for x in games['Team']]
    df['Date'] = games['Date']
    df['Opponent'] = games['Opponent']
    df['Spread'] = games['P_S']
    df['Win_Loss'] = games['Wins']
    df['Home'] = games['Home']
    df['Away'] = games['Away']
    
    # GP = Games Played prior to this game
    df['GP'] = games.groupby('Team')['Team'].transform(lambda x: x.expanding().count()-1)
    # get prior average stats for each team before the game
    for i in ['Wins','Tm', 'Opp', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'ORtg', 'DRtg', 'Pace', 'FTr',
       '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'OeFG%', 'OTOV%', 'ORB%',
       'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA','P_S', 'FG_S', 'FGA_S', 'FG%_S', '3P_S', '3PA_S',
       '3P%_S', 'FT_S', 'FTA_S', 'FT%_S', 'TRB_S', 'AST_S', 'STL_S', 'BLK_S','TOV_S', 'PF_S', 'FTr_S', '3PAr_S', 'TS%_S', 'TRB%_S', 'AST%_S',
       'STL%_S', 'BLK%_S']:
        df[i] = games.groupby('Team')[i].transform(lambda x: x.expanding().sum()-x)/df['GP']
    return df

## Append (vs) Opponent Stats 

In [11]:
def vs_df(df):
    '''
    This will compute the running stats for teams, with season-to-date info prior to each game
    '''
    # GP = Games Played prior to this game
    winners = df[df.Win_Loss == 1]
    losers = df[df.Win_Loss == 0]
    
    winner_merge = pd.merge(winners, losers, left_on='url',right_on='url',how='outer',suffixes=('','_vs'))
    loser_merge = pd.merge(losers, winners, left_on='url',right_on='url',how='outer',suffixes=('','_vs'))
    final = pd.concat([winner_merge,loser_merge])
    return final

In [41]:
def remove_dups(df):
    '''
    remove the second instance of each game, where the teams are just flipped
    
    '''
    df_urlsort = df.sort_values('url')
    
    total = len(df_urlsort)
    unique = len(df_urlsort.url.unique())
    check = (total/unique == 2) 
    print(f'total games: {total} unique games: {unique} check: {check}')
    
    df_final = df_urlsort.iloc[range(0,len(df_urlsort),2),]
    total = len(df_final)
    unique = len(df_final.url.unique())
    check = (total == unique) 
    print(f'games kept: {total} unique games: {unique} check: {check}')
    
    return df_final

In [29]:
my_seasons = [2014,2015,2016,2017,2018,2019]
for i in my_seasons:
    team_lookup = pickle.load(open(f'team_lookup_{i}.pickle', "rb"))
    combo_log = pickle.load(open(f'combo_log_{i}.pickle', "rb"))
    combo_log = clean_gamelog(combo_log)
    combo_log = helper_columns(combo_log,team_lookup)
    combo_log = D1_only(combo_log, team_lookup) #removes non-D1 opponents
    combo_log = calc_spreads(combo_log) #calculates the spreads between team and opponent for each game
    with open(f'clean_combo_{i}.pickle', 'wb') as handle:
        pickle.dump(combo_log, handle, protocol=pickle.HIGHEST_PROTOCOL)
    df = agg_df(combo_log) #calculates running season to date info for each game
    df = vs_df(df) #adds opponent info
    df = remove_dups(df)
    with open(f'final_{i}.pickle', 'wb') as handle:
        pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)    

total games: 11016 unique games: 5508 check: True
games kept: 5508 unique games: 5508 check: True
total games: 10998 unique games: 5499 check: True
games kept: 5499 unique games: 5499 check: True
total games: 11032 unique games: 5516 check: True
games kept: 5516 unique games: 5516 check: True
total games: 11070 unique games: 5535 check: True
games kept: 5535 unique games: 5535 check: True
total games: 11078 unique games: 5539 check: True
games kept: 5539 unique games: 5539 check: True
total games: 11206 unique games: 5603 check: True
games kept: 5603 unique games: 5603 check: True


In [42]:
my_seasons = [2014,2015,2016,2017,2018,2019]
for i in my_seasons:
    clean_combo_log = pickle.load(open(f'clean_combo_{i}.pickle', "rb"))
    df = agg_df(clean_combo_log) #calculates running season to date info for each game
    df = vs_df(df) #adds opponent info
    df = remove_dups(df)
    with open(f'final_{i}.pickle', 'wb') as handle:
        pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)    


total games: 11016 unique games: 5508 check: True
games kept: 5508 unique games: 5508 check: True
total games: 10998 unique games: 5499 check: True
games kept: 5499 unique games: 5499 check: True
total games: 11032 unique games: 5516 check: True
games kept: 5516 unique games: 5516 check: True
total games: 11070 unique games: 5535 check: True
games kept: 5535 unique games: 5535 check: True
total games: 11078 unique games: 5539 check: True
games kept: 5539 unique games: 5539 check: True
total games: 11206 unique games: 5603 check: True
games kept: 5603 unique games: 5603 check: True


## Adding line information - FUTURE WORK
below code can be used as starter but is not functional. sometimes gets the sign wrong. would need to correct for that

In [3]:
lines = pickle.load(open('lines.pickle', "rb"))
lines.date = pd.to_datetime(lines.date)
lines.reset_index(drop=True)
line_school_dict = pd.read_csv('line_school_dict.csv')
school_lookup = dict(zip(line_school_dict.Lines,line_school_dict.Log))

home_fix = []
for i in range(len(lines)):
    try:
        home_fix.append(school_lookup[lines.iloc[i,1]])
    except KeyError:
        home_fix.append(None)
        
lines['home']=home_fix
small_lines = lines.loc[:,['date','home','line']]

In [12]:
my_seasons = [2014,2015,2016,2017,2018,2019]
for i in my_seasons:
 
    df = pickle.load(open(f'final_{i}.pickle', "rb"))
    df = pd.merge(df, small_lines, how = 'left', left_on = ['Date','Team'], right_on = ['date','home'], right_index=False)
    df = pd.merge(df, small_lines, how = 'left', left_on = ['Date','Opponent'], right_on = ['date','home'], right_index=False)
    df = df.drop(columns=['date_x','home_x','date_y','home_y'])
    line = df.line_x.combine_first(df.line_y)
    df['line'] = line
    df = df.drop(columns=['line_x', 'line_y'])   
    with open(f'final_{i}.pickle', 'wb') as handle:
        pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

url                 object
Team                object
Date        datetime64[ns]
Opponent            object
Spread               int64
                 ...      
line_x             float64
line_y             float64
line_x             float64
line_y             float64
line               float64
Length: 142, dtype: object


AttributeError: 'DataFrame' object has no attribute 'dtype'

In [13]:
lines

,date,home,hscore,road,rscore,line,lineavg,linesag,linesage,linesagp,...,linepib,line7ot,lineer,linedd,linemassey,lineespn,linedunk,lineround,lineteamrnks,linetalis
0,2013-11-08,Army,68.0,Air Force,79.0,3.5,-2.16,-0.97,-0.97,-0.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-11-08,NC State,98.0,Appalachian St.,77.0,14.0,18.23,16.78,16.78,16.78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-11-08,Providence,82.0,Boston College,78.0,3.0,3.55,3.50,3.50,3.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-11-08,Texas A&M,82.0,Buffalo,58.0,6.0,7.65,5.68,5.68,5.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-11-08,Arizona,73.0,Cal Poly SLO,62.0,16.5,18.89,17.58,17.58,17.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,2019-04-05,DePaul,NaN,South Florida,1.0,1.0,3.23,2.28,2.48,2.25,...,NaN,3.0,NaN,NaN,3.26,NaN,4.77,2.0,2.1,3.4
5560,2019-04-05,DePaul,NaN,South Florida,1.0,1.0,3.00,2.28,2.48,2.25,...,NaN,3.0,NaN,NaN,3.26,NaN,4.77,2.0,2.1,0.1
5561,2019-04-06,Michigan State,NaN,Texas Tech,2.0,2.0,1.72,2.93,0.50,3.03,...,NaN,1.0,NaN,NaN,1.89,NaN,2.19,1.0,2.1,2.9
5562,2019-04-06,Virginia,NaN,Auburn,6.0,6.0,4.02,4.84,-4.13,4.89,...,NaN,8.0,NaN,NaN,3.01,NaN,5.30,5.0,5.0,5.3


In [15]:
lines['actual'] = lines.hscore-lines.rscore

In [18]:
lines['error'] = abs(lines.line-lines.actual)

In [21]:
np.mean(lines.line)

4.9306625577812015